In [13]:
from tensorflow_docs.vis import embed  # TensorFlow 문서의 시각화를 위한 도구
from tensorflow import keras  # TensorFlow의 고수준 신경망 API
from imutils import paths  # 이미지 처리를 위한 유틸리티 함수들을 제공

import time
import matplotlib.pyplot as plt  # 그래프와 이미지를 시각화하기 위한 라이브러리
import tensorflow as tf  # TensorFlow 라이브러리, 딥러닝 모델을 구성하고 훈련하기 위해 사용
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리
import numpy as np  # 수치 계산을 위한 라이브러리
import imageio  # 이미지 읽기/쓰기를 위한 라이브러리
import cv2  # OpenCV 라이브러리, 이미지 및 비디오 처리를 위해 사용
import os  # 운영체제와 상호작용을 위한 라이브러리
import math
import mediapipe as mp

In [5]:
dataset_path = os.listdir('video/dataset/train')
label_types = os.listdir('video/dataset/train')
# 훈련 데이터셋을 위한 비어있는 리스트 초기화
rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/train' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('video/dataset/train'+'/'+item)    
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('video/dataset/train'+'/'+item)+'/'+room))
# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name']).loc[:,['video_name','tag']]
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('train.csv', encoding='utf-8-sig')
df

,video_name,tag
0,video/dataset/train/가볍다/1.mp4,가볍다
1,video/dataset/train/가져오다/2.mp4,가져오다
2,video/dataset/train/가짜/3.mp4,가짜
3,video/dataset/train/가치/4.mp4,가치
4,video/dataset/train/보관/5.mp4,보관
5,video/dataset/train/보내다/6.mp4,보내다
6,video/dataset/train/보다/7.mp4,보다
7,video/dataset/train/안경/8.mp4,안경
8,video/dataset/train/알다/9.mp4,알다
9,video/dataset/train/월요일/10.mp4,월요일


In [6]:
dataset_path = os.listdir('video/dataset/test')
room_types = os.listdir('video/dataset/test')

rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/test' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('video/dataset/test'+'/'+item)
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('video/dataset/test'+'/'+item)+'/'+room))

# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name'])
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('test.csv', encoding='utf-8-sig')
df

,video_name,tag
0,video/dataset/test/가볍다/1.mp4,가볍다
1,video/dataset/test/가져오다/2.mp4,가져오다
2,video/dataset/test/가짜/3.mp4,가짜
3,video/dataset/test/가치/4.mp4,가치
4,video/dataset/test/보관/5.mp4,보관
5,video/dataset/test/보내다/6.mp4,보내다
6,video/dataset/test/보다/7.mp4,보다
7,video/dataset/test/안경/8.mp4,안경
8,video/dataset/test/알다/9.mp4,알다
9,video/dataset/test/월요일/10.mp4,월요일


In [7]:
# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
# GPU가 사용 가능한 경우
if gpus:
    try:
        # 첫 번째 GPU에 대해 메모리 제한 설정
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], 
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]
        )
    except RuntimeError as e:
        print(e)

In [8]:
# 훈련 데이터셋과 테스트 데이터셋을 각각 CSV 파일에서 로드
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 훈련 및 테스트 데이터셋 크기 출력
print(f"Total video for training: {len(train_df)}")
print(f"Total video for testing: {len(test_df)}")

# 훈련 데이터셋의 샘플 5개 출력
train_df.sample(5)

Total video for training: 10
Total video for testing: 10


,Unnamed: 0,video_name,tag
6,6,video/dataset/train/보다/7.mp4,보다
9,9,video/dataset/train/월요일/10.mp4,월요일
4,4,video/dataset/train/보관/5.mp4,보관
0,0,video/dataset/train/가볍다/1.mp4,가볍다
1,1,video/dataset/train/가져오다/2.mp4,가져오다


In [21]:
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

In [22]:
# 이미지의 크기, 배치 크기, 에포크 수를 정의하는 하이퍼파라미터입니다.
IMG_SIZE = 224  # 입력 이미지의 크기를 정의합니다.
BATCH_SIZE = 64  # 한 번에 처리할 이미지의 수를 정의합니다.
EPOCHS = 10  # 모델을 훈련할 때 전체 데이터셋을 반복할 횟수를 정의합니다.

# 비디오 처리에 사용할 최대 시퀀스 길이와 특징 벡터의 크기를 정의합니다.
MAX_SEQ_LENGTH = 20  # 처리할 비디오의 최대 프레임 수를 정의합니다.
NUM_FEATURES = 2048  # 비디오 프레임에서 추출할 특징의 차원 수를 정의합니다.

In [23]:
# 주어진 이미지에서 중앙에 맞춰 정사각형으로 잘나내는 함수
def crop_center_square(frame):
    # 이미지의 높이(y)와 너비(x)를 가져옴
    y, x = frame.shape[0:2]
    # 이미지의 높이와 너비 중 더 작은 값을 선택하여 정사각형의 크기를 결정
    min_dim = min(y, x)
    # 정사각형을 이미지 중앙에 위치시키기 위해 시작점의 x좌표와 y좌표를 계산
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    # 계산된 시작점과 정사각형의 크기를 이용하여 이미지의 중앙 부분을 잘라냅니다.
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [24]:
# 비디오 파일을 로드하고, 각 프레임을 처리하여 배열로 반환하는 함수
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    # OpenCV를 사용하여 비디오 파일 열기
    cap = cv2.VideoCapture(path)
    frames = []
    skeletons = []  # 스켈레톤 데이터
    hand_landmarks = []  # 손 데이터
    face_landmarks = []  # 얼굴 데이터
    try:
        while True:            
            ret, frame = cap.read() # 비디오에서 프레임을 하나씩 읽기            
            if not ret:
                break # 읽을 프레임이 없으면 반복문을 종료
            frame = crop_center_square(frame)             # 읽은 프레임에서 중앙의 정사각형 부분을 잘라냄            
            frame = cv2.resize(frame, resize)            # 프레임의 크기를 지정된 크기로 조절
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)            
            # Mediapipe를 사용하여 스켈레톤 추출
            hands_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)
            face_results = face_detection.process(frame_rgb)
           
            if pose_results.pose_landmarks:
                skeletons.append(pose_results.pose_landmarks.landmark)
            if hands_results.multi_hand_landmarks:
                hand_landmarks.append(hands_results.multi_hand_landmarks)
            if face_results.detections:
                face_landmarks.append(face_results.detections)
            
            # OpenCV는 BGR 색상 순서를 사용하므로, 이를 RGB 순서로 변경
            frame = frame[:, :, [2, 1, 0]]
            # 처리된 프레임을 프레임 리스트에 추가
            frames.append(frame)
            # max_frames가 지정된 경우, 지정된 수의 프레임만큼만 처리
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 파일을 닫기
        cap.release()
        pose.close
        hands.close
        face_detection.close
    # 처리된 모든 프레임을 numpy 배열로 변환하여 반환
    return np.array(frames), skeletons, hand_landmarks, face_landmarks

In [25]:
# 특징추출 함수
def build_feature_extractor():
    # 이미지 특징 추출을 위한 InceptionV3 모델
    # InceptionV3 모델을 특징 추출기로 사용.
    # ImageNet 데이터로 사전 훈련된 가중치를 사용하고, 최상위 층은 포함하지 않으며, 평균 풀링을 사용
    base_model = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    # InceptionV3에 맞게 입력 데이터를 전처리하는 함수
    preprocess_input = keras.applications.inception_v3.preprocess_input
    # 모델의 입력을 정의
    image_input = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    # 입력 데이터를 전처리
    preprocessed_image = preprocess_input(image_input)
    image_features = base_model(preprocessed_image)

    # Mediapipe 데이터를 위한 입력 레이어 및 처리 레이어
    # 예시로, Mediapipe 데이터의 차원을 상정하여 입력 레이어를 정의
    mediapipe_input = keras.Input((1599+63,))
    mediapipe_features = keras.layers.Dense(128, activation="relu")(mediapipe_input)

    # 이미지 특징과 Mediapipe 데이터의 결합
    combined_features = keras.layers.concatenate([image_features, mediapipe_features])

    # 최종 모델
    outputs = keras.layers.Dense(10, activation="softmax")(combined_features)
    return keras.Model(inputs=[image_input, mediapipe_input], outputs=outputs, name="feature_extractor")

In [26]:
# 특징 추출기 모델을 생성
feature_extractor = build_feature_extractor()

In [27]:
# 클래스 라벨을 정수로 변환하는 레이어를 생성
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# 라벨 프로세서가 이해하는 어휘목록을 출력
print(label_processor.get_vocabulary())

['가볍다', '가져오다', '가짜', '가치', '보관', '보내다', '보다', '안경', '알다', '월요일']


In [28]:
# 주어진 데이터프레임(df)과 루트 디렉터리(root_dir)를 사용하여 모든 비디오를 준비하는 함수입니다.
def prepare_all_videos(df, root_dir):
    num_samples = len(df)  # 샘플의 수를 결정합니다.
    video_paths = df["video_name"].values.tolist()  # 비디오 경로를 리스트로 변환합니다.
    labels = df["tag"].values  # 라벨 값을 가져옵니다.
    labels = label_processor(labels[..., None]).numpy()  # 라벨을 처리하여 넘파이 배열로 변환합니다.

    # `frame_masks`와 `frame_features`는 우리의 순차 모델에 제공될 데이터입니다.
    # `frame_masks`는 시간 단계가 패딩으로 마스킹되었는지 여부를 나타내는 부울 값들을 포함합니다.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # 각 비디오에 대해서.
    for idx, path in enumerate(video_paths):
        # 모든 프레임을 수집하고 배치 차원을 추가합니다.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # 현재 비디오의 마스크와 특징을 저장하기 위한 임시 공간을 초기화합니다.
        temp_frame_mask = np.zeros(
            shape=(
                1,
                MAX_SEQ_LENGTH,
            ),
            dtype="bool",
        )
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # 현재 비디오의 프레임에서 특징을 추출합니다.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                # 특징 추출기를 사용하여 각 프레임의 특징을 예측합니다.
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            # 마스크를 설정합니다. 1 = 마스킹되지 않음, 0 = 마스킹됨
            temp_frame_mask[i, :length] = 1  

        # 임시 특징과 마스크를 각각의 특징 및 마스크 배열에 할당합니다.
        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    # 특징과 마스크 배열, 라벨을 반환합니다.
    return (frame_features, frame_masks), labels

# 훈련 데이터와 라벨을 준비합니다.
train_data, train_labels = prepare_all_videos(train_df, "train")
# 테스트 데이터와 라벨을 준비합니다.
test_data, test_labels = prepare_all_videos(test_df, "test")

# 훈련 데이터 세트의 프레임 특징과 마스크의 차원을 출력합니다.
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

TypeError: tuple indices must be integers or slices, not tuple